<a href="https://colab.research.google.com/github/lorenaps0/Semantic_search_engine/blob/main/Semantic_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando sdk do Google

In [1]:
!pip install -q -U google-generativeai

Importações e configurações iniciais

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY=('COLE SUA API KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
   if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Listagem dos documentos que serão buscados

In [12]:
DOCUMENT1 = {
      "Título": "A Importância do Sono",
      "Conteúdo": "Dormir não é apenas descansar o corpo, mas também essencial para o bom funcionamento do cérebro. Durante o sono, o cérebro consolida memórias, organiza informações e se recupera do estresse do dia. Privação de sono pode afetar a concentração, o humor e até mesmo o sistema imunológico."
}

DOCUMENT2 = {
    "Título": "A Arte Milenar do Bonsai",
    "Conteúdo": "Originária da China, a arte do bonsai consiste em cultivar miniaturas de árvores em vasos, criando verdadeiras esculturas vivas. Através de técnicas de poda, aramação e controle de raízes, o bonsai exige paciência, dedicação e sensibilidade para moldar a natureza em formas de grande beleza e harmonia."
}

DOCUMENT3 = {
   "Título": "O Poder da Música",
   "Conteúdo": "A música tem o poder de despertar emoções, conectar pessoas e transcender barreiras culturais. Ela pode nos energizar, acalmar, inspirar e transportar para outras dimensões. Escutar música estimula áreas do cérebro relacionadas à memória, linguagem e coordenação motora. Seja qual for o estilo musical, a música enriquece a vida e nos conecta com a beleza da expressão artística."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [18]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,A Importância do Sono,"Dormir não é apenas descansar o corpo, mas tam..."
1,A Arte Milenar do Bonsai,"Originária da China, a arte do bonsai consiste..."
2,O Poder da Música,"A música tem o poder de despertar emoções, con..."


In [19]:
model = 'models/embedding-001'

In [20]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type='RETRIEVAL_DOCUMENT')['embedding']

In [25]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,A Importância do Sono,"Dormir não é apenas descansar o corpo, mas tam...","[-0.009183546, -0.02694537, -0.047302943, 0.03..."
1,A Arte Milenar do Bonsai,"Originária da China, a arte do bonsai consiste...","[0.006907176, -0.024027415, 0.0022740283, 0.02..."
2,O Poder da Música,"A música tem o poder de despertar emoções, con...","[-0.0067312373, 0.0027926853, 0.0027613016, 0...."


In [32]:
def gerar_e_buscar_consulta(consulta, base, model):

    embedding_da_consulta = genai.embed_content(model=model,
                                                  content=consulta,
                                                  task_type="RETRIEVAL_QUERY")["embedding"]

    produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

    indice = np.argmax(produtos_escalares)

    return df.iloc[indice]["Conteudo"]

In [33]:
consulta = "O que acontece durante o sono?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Dormir não é apenas descansar o corpo, mas também essencial para o bom funcionamento do cérebro. Durante o sono, o cérebro consolida memórias, organiza informações e se recupera do estresse do dia. Privação de sono pode afetar a concentração, o humor e até mesmo o sistema imunológico.


In [35]:
generation_config = {
    'candidate_count': 1,
    'temperature': 0.5
}

In [37]:
prompt = f"Agora reescreva o texto de forma mais descontraida, sem adicionar informações que não façam parte do texto: {trecho}"

model1_2 = genai.GenerativeModel("gemini-1.0-pro",
                                 generation_config=generation_config,)
response = model1_2.generate_content(prompt)
print(response.text)

Dormir não é só pra dar uma folga pro corpo, é também pra dar uma geral no cérebro! Enquanto você dorme, seu cérebro tá lá, juntando as peças do quebra-cabeça, organizando as ideias e dando um tempo no estresse do dia.

Se você fica sem dormir, é tipo um carro sem gasolina: sua concentração vai pro saco, seu humor fica na bad e até seu sistema imunológico pode dar uma mancada. Então, não vacila, dorme bem pra manter seu cérebro em dia!
